### Python script with Weighted Hamming Kernel 


- A variation of the Hamming kernel where different features (members) have different importance.
- **Kernel function:**

$
k(x, x') = \exp \left( - \sum_i w_i \mathbf{1} (x_i \neq x'_i) \right)
$

where $ w_i $ is a weight indicating the relative importance of design variable $ i $.

### Example Use:
Some truss members might be more critical than others. The weighted Hamming kernel allows prioritizing important members in the optimization process.


In [1]:
import numpy as np
from sklearn.gaussian_process import GaussianProcessRegressor

def weighted_hamming_kernel(X, Y=None, weights=None):
    """
    Weighted Hamming Kernel for binary variables.
    
    Parameters:
        X: ndarray (n_samples_X, n_features) - Binary input data.
        Y: ndarray (n_samples_Y, n_features) - Binary input data (optional).
        weights: ndarray (n_features,) - Feature weights (optional).
    
    Returns:
        Kernel matrix K (n_samples_X, n_samples_Y)
    """
    X = np.atleast_2d(X)
    Y = X if Y is None else np.atleast_2d(Y)

    if weights is None:
        weights = np.ones(X.shape[1])  # Default: equal weights

    # Compute weighted Hamming distances
    dists = np.sum(weights * (X[:, None, :] != Y[None, :, :]), axis=2)

    # Apply exponential transformation to obtain kernel matrix
    return np.exp(-dists)

# Example: Binary truss topology optimization

# Binary design space (each row represents a binary vector for truss topology)
X_train = np.array([
    [1, 0, 1, 1, 0], 
    [0, 1, 1, 0, 1], 
    [1, 1, 0, 1, 0],
    [0, 0, 1, 1, 1]
])

# Simulated performance scores (e.g., compliance or weight minimization objective)
y_train = np.array([0.8, 0.6, 0.9, 0.7])  # Lower is better

# Define weights for the kernel (important members have higher weight)
weights = np.array([1.5, 1.0, 0.5, 1.2, 0.8])

# Create a Gaussian Process model
gp = GaussianProcessRegressor(alpha=0.01, normalize_y=True)

# Custom kernel function for GPR
def custom_kernel(X, Y=None):
    return weighted_hamming_kernel(X, Y, weights=weights)

# Fit GP model using manually computed kernel matrix
gp.fit(X_train, y_train)

# Define acquisition function (Expected Improvement)
def acquisition(X_new, gp, X_train, y_train):
    """Expected Improvement Acquisition Function"""
    mu, sigma = gp.predict(X_new, return_std=True)
    sigma = np.maximum(sigma, 1e-6)  # Numerical stability
    best_y = np.min(y_train)
    improvement = best_y - mu
    return improvement * (improvement > 0)  # EI only for improving points

# Bayesian Optimization loop: Find best topology
def optimize_topology(X_train, y_train, gp, num_iters=1000):
    best_topology = None
    best_value = float('inf')

    for _ in range(num_iters):
        X_candidates = np.random.randint(0, 2, (10, 5))  # Generate random binary candidates
        EI_values = acquisition(X_candidates, gp, X_train, y_train)
        
        best_idx = np.argmax(EI_values)
        candidate = X_candidates[best_idx]

        # Evaluate candidate (simulated performance score)
        candidate_score = np.random.uniform(0.5, 1.0)

        if candidate_score < best_value:
            best_topology, best_value = candidate, candidate_score

        # Update dataset
        X_train = np.vstack([X_train, candidate])
        y_train = np.append(y_train, candidate_score)

        # Re-train GP
        gp.fit(X_train, y_train)

    return best_topology, best_value

# Run optimization
best_truss, best_score = optimize_topology(X_train, y_train, gp)
print("Best Truss Design:", best_truss)
print("Best Score:", best_score)

Best Truss Design: [0 1 0 0 0]
Best Score: 0.500660013179572


### Python Script with Tanimoto Kernel

- Used for binary and discrete data.
- Measures the similarity of two binary vectors using the **Tanimoto coefficient**:

$
k(x, x') = \frac{x \cdot x'}{||x||^2 + ||x'||^2 - x \cdot x'}
$

This ensures that if two designs share more members, they are considered more similar.

### Example Use:
Comparing different truss topologies by the proportion of common structural members.

In [5]:
import numpy as np
from sklearn.gaussian_process import GaussianProcessRegressor

def tanimoto_kernel(X, Y=None):
    """
    Tanimoto Kernel for binary variables.
    
    Parameters:
        X: ndarray (n_samples_X, n_features) - Binary input data.
        Y: ndarray (n_samples_Y, n_features) - Binary input data (optional).
    
    Returns:
        Kernel matrix K (n_samples_X, n_samples_Y)
    """
    X = np.atleast_2d(X)
    Y = X if Y is None else np.atleast_2d(Y)

    # Compute dot product (X · Y)
    XY = np.dot(X, Y.T)

    # Compute squared norms ||X||^2 and ||Y||^2
    X_norm = np.sum(X**2, axis=1).reshape(-1, 1)
    Y_norm = np.sum(Y**2, axis=1).reshape(1, -1)

    # Compute Tanimoto Kernel
    K = XY / (X_norm + Y_norm - XY + 1e-6)  # Small epsilon for numerical stability

    return K

# Example: Binary truss topology optimization

# Binary design space (each row represents a binary vector for truss topology)
X_train = np.array([
    [1, 0, 1, 1, 0], 
    [0, 1, 1, 0, 1], 
    [1, 1, 0, 1, 0],
    [0, 0, 1, 1, 1]
])

# Simulated performance scores (e.g., compliance or weight minimization objective)
y_train = np.array([0.8, 0.6, 0.9, 0.7])  # Lower is better

# Create a Gaussian Process model
gp = GaussianProcessRegressor(alpha=0.01, normalize_y=True)

# Custom kernel function for GPR
def custom_kernel(X, Y=None):
    return tanimoto_kernel(X, Y)

# Fit GP model using manually computed kernel matrix
gp.fit(X_train, y_train)

# Define acquisition function (Expected Improvement)
def acquisition(X_new, gp, X_train, y_train):
    """Expected Improvement Acquisition Function"""
    mu, sigma = gp.predict(X_new, return_std=True)
    sigma = np.maximum(sigma, 1e-6)  # Numerical stability
    best_y = np.min(y_train)
    improvement = best_y - mu
    return improvement * (improvement > 0)  # EI only for improving points

# Bayesian Optimization loop: Find best topology
def optimize_topology(X_train, y_train, gp, num_iters=1000):
    best_topology = None
    best_value = float('inf')

    for _ in range(num_iters):
        X_candidates = np.random.randint(0, 2, (10, 5))  # Generate random binary candidates
        EI_values = acquisition(X_candidates, gp, X_train, y_train)
        
        best_idx = np.argmax(EI_values)
        candidate = X_candidates[best_idx]

        # Evaluate candidate (simulated performance score)
        candidate_score = np.random.uniform(0.5, 1.0)

        if candidate_score < best_value:
            best_topology, best_value = candidate, candidate_score

        # Update dataset
        X_train = np.vstack([X_train, candidate])
        y_train = np.append(y_train, candidate_score)

        # Re-train GP
        gp.fit(X_train, y_train)

    return best_topology, best_value

# Run optimization
best_truss, best_score = optimize_topology(X_train, y_train, gp)
print("Best Truss Design:", best_truss)
print("Best Score:", best_score)

Best Truss Design: [0 0 1 0 1]
Best Score: 0.5000415384724269


In [2]:
import numpy as np
from hyperopt import hp, fmin, tpe, Trials

# Number of structural members
NUM_MEMBERS = 10

# Define the search space (binary variables for topology selection)
search_space = [hp.choice(f'beam_{i}', [0, 1]) for i in range(NUM_MEMBERS)]

# Simulated Structural Analysis Function (FEA proxy)
def evaluate_topology(config):
    """
    Evaluates the weight and structural efficiency of a gridshell topology.
    The function returns an objective value where lower is better.
    """
    weight_per_member = 10  # Assume each member has a weight of 10 kg
    selected_members = np.array(config)  # Convert list to numpy array
    
    # Calculate total weight
    total_weight = np.sum(selected_members) * weight_per_member

    # Simulated "stiffness penalty" (more members = better stiffness)
    stiffness_penalty = np.sum(selected_members)  # More members → better stability

    # Objective: Minimize weight, but penalize for lack of stiffness
    if stiffness_penalty == 0:
        return 1000  # High penalty for empty structures
    
    performance_metric = total_weight - 5 * stiffness_penalty  # Balance weight & stiffness
    return performance_metric  # Lower is better

# Run Bayesian Optimization (TPE)
trials = Trials()
best_topology = fmin(
    fn=evaluate_topology,  # Objective function
    space=search_space,  # Search space
    algo=tpe.suggest,  # Use TPE optimizer
    max_evals=50,  # Number of iterations
    trials=trials  # Store results
)

# Decode best topology (convert indices to binary)
best_structure = [best_topology[f'beam_{i}'] for i in range(NUM_MEMBERS)]

# Print results
print("\nBest Topology (Binary Vector):", best_structure)
print("Total Weight (kg):", np.sum(best_structure) * 10)

100%|██████████| 50/50 [00:00<00:00, 105.50trial/s, best loss: 5.0] 

Best Topology (Binary Vector): [np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(1), np.int64(0), np.int64(0), np.int64(0)]
Total Weight (kg): 10
